In [1]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm_notebook as tqdm
from joblib import Parallel, delayed


In [2]:
db = pickle.load(open( "db.pkl", "rb" ))

In [3]:
placement_hist = pickle.load(open("plot_hist.pkl","rb"))

In [16]:
db.num_physical_nodes + db.num_filler_nodes

211447

In [ ]:
# first n nodes are physical. Last are filler.
# placement_hist is a list of [x:y] positions

In [ ]:
from draw_place import DrawPlace

In [ ]:
d = DrawPlace(db)

In [ ]:
d.forward(placement_hist[0],'test.png')

In [ ]:
placement_hist[100]

In [4]:
id2name = {v: k for k, v in db.node_name2id_map.items()}

In [12]:
len(db.node_name2id_map.items())

211447

In [17]:
# too slow
placement_hist_dfs = []
for i in tqdm(range(db.num_nodes)):
    xpos = []
    ypos = []
    placement_df = pd.DataFrame(columns=['id','x','y'])
    if i <= db.num_physical_nodes:
        id_col = [id2name[i]]*len(placement_hist)
    else: 
        id_col = "f"+i
    for placement in placement_hist:
        xpos.append(placement[i])
        ypos.append(placement[db.num_nodes + i])
    placement_df['id'] = id_col
    placement_df['x'] = xpos
    placement_df['y'] = ypos
    placement_hist_dfs.append(placement_df)
    
placement_hist_df = pd.concatenate(placement_hist_dfs)

KeyboardInterrupt: 

In [ ]:
np.stack(placement_hist).shape

In [ ]:
np.stack(placement_hist)[:,1]

In [19]:
# still slow
placement_mat = np.stack(placement_hist)
for i in tqdm(range(db.num_nodes)):
    placement_df = pd.DataFrame(columns=['id','x','y'])
    if i <= db.num_physical_nodes:
        id_col = [id2name[i]]*len(placement_hist)
    else: 
        id_col = "f"+i    
    xpos = placement_mat[:,i]
    ypos = placement_mat[:,db.num_nodes + i]
    placement_df['id'] = id_col
    placement_df['x'] = xpos
    placement_df['y'] = ypos
    placement_hist_dfs.append(placement_df)
    
placement_hist_df = pd.concatenate(placement_hist_dfs)

KeyboardInterrupt: 

In [78]:
# multithread
placement_mat = np.stack(placement_hist)
def get_pl_df(placement_mat,i):
    #placement_df = pd.DataFrame(columns=['id','x','y'])
    placement_dict = {}
    if i < db.num_physical_nodes:
        id_col = [id2name[i]]*placement_mat.shape[0]
    else: 
        id_col = ["f"+str(i)]*placement_mat.shape[0]
    xpos = placement_mat[:,i]
    ypos = placement_mat[:,db.num_nodes + i]
    placement_dict['id'] = id_col
    placement_dict['x'] = xpos
    placement_dict['y'] = ypos
    #return pd.DataFrame.from_dict(placement_dict)
    return placement_dict

In [79]:
r = Parallel(n_jobs=-1, verbose=1, backend="threading")(delayed(get_pl_df)(placement_mat,i) for i in range(db.num_nodes))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 11234 tasks  

In [84]:
%%time
X = pd.concat(r)
X = pd.DataFrame(r)

CPU times: user 44.4 s, sys: 886 ms, total: 45.3 s
Wall time: 45.3 s


In [93]:
%%time
traj_list = [c[1][["x", "y"]].values for c in X.groupby("id")]

KeyboardInterrupt: 

In [ ]:
ts = time.time()
p_dist = tdist.pdist(traj_list, metric="sspd")
nb_dist = len(p_dist)
te = time.time()
print("%d Distances computed in %d seconds" % (nb_dist, te-ts))
np.save(output_file, p_dist)